In [ ]:
## Recommendation System with LightFM Practice
## https://towardsdatascience.com/solving-business-usecases-by-recommender-system-using-lightfm-4ba7b3ac8e62

In [5]:
from recsys import * ## Recommender system cookbook
from generic_preprocessing import * ## pre-processing code
from IPython.display import HTML ## setting display options for IPython Notebook
import pandas as pd


In [10]:
# Importing ratings data 
ratings = pd.read_csv('Data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
# Importing movie data
movies = pd.read_csv('Data/movies.csv')
movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
# Creating interaction matrix using rating data
interactions = create_interaction_matrix(df=ratings, 
                                         user_col = 'userId', 
                                        item_col = 'movieId', 
                                        rating_col = 'rating')
interactions.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Create User Dict
user_dict = create_user_dict(interactions=interactions)
# Create Item dict
movies_dict = create_item_dict(df = movies,
                               id_col = 'movieId',
                               name_col = 'title')

In [20]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [21]:
# Usecase 1 

In [27]:
## Calling 10 movie recommendation for user id 11
rec_list = sample_recommendation_user(model = mf_model, 
                                      interactions = interactions, 
                                      user_id = 11, 
                                      user_dict = user_dict,
                                      item_dict = movies_dict, 
                                      threshold = 4,
                                      nrec_items = 10,
                                      show = True)

Known Likes:
1- Saving Private Ryan (1998)
2- As Good as It Gets (1997)
3- Titanic (1997)
4- Amistad (1997)
5- Contact (1997)
6- Last of the Mohicans, The (1992)
7- Top Gun (1986)
8- Silence of the Lambs, The (1991)
9- Searching for Bobby Fischer (1993)
10- Fugitive, The (1993)
11- Forrest Gump (1994)
12- Clear and Present Danger (1994)
13- Apollo 13 (1995)
14- Braveheart (1995)
15- Heat (1995)

 Recommended Items:
1- Terminator 2: Judgment Day (1991)
2- Cliffhanger (1993)
3- Speed (1994)
4- Batman (1989)
5- Jurassic Park (1993)
6- Crimson Tide (1995)
7- Independence Day (a.k.a. ID4) (1996)
8- Die Hard: With a Vengeance (1995)
9- True Lies (1994)
10- Pulp Fiction (1994)


In [25]:
print(rec_list)

[589, 434, 377, 592, 480, 161, 780, 165, 380, 296]


In [37]:
## Calling 15 user recommendatuion for item id 1
sample_recommendation_item(model = mf_model,
                          interactions = interactions,
                          item_id =1,
                          user_dict =user_dict,
                          item_dict= movies_dict,
                          number_of_user = 15)

[399, 173, 235, 549, 126, 394, 557, 189, 569, 81, 379, 8, 507, 133, 347]

In [40]:
## Creating item-item distance matrix // Similarity
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model,
                                     interactions = interactions)
## Checking item embedding distance matrix
item_item_dist.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.792902,0.419435,-0.261133,0.388790,0.468722,0.279629,0.267107,0.106813,0.742607,...,-0.518225,-0.598297,-0.479022,-0.582091,-0.557749,-0.473429,-0.582529,-0.580725,-0.600257,-0.331548
2,0.792902,1.000000,0.487280,-0.136567,0.518397,0.443920,0.371002,0.370085,0.143824,0.824195,...,-0.473643,-0.565293,-0.405311,-0.546880,-0.540127,-0.465355,-0.557515,-0.529496,-0.531313,-0.149780
3,0.419435,0.487280,1.000000,0.270626,0.604637,0.444199,0.561326,0.466722,0.601784,0.592016,...,-0.320180,-0.313743,-0.247040,-0.282181,-0.422287,-0.311102,-0.386105,-0.361101,-0.391437,-0.513933
4,-0.261133,-0.136567,0.270626,1.000000,0.288765,-0.166056,0.422381,0.355903,0.353909,-0.098202,...,0.325813,0.391577,0.327562,0.366045,0.347466,0.375027,0.364509,0.384939,0.441632,-0.039257
5,0.388790,0.518397,0.604637,0.288765,1.000000,0.345218,0.686092,0.412421,0.411401,0.417466,...,-0.401362,-0.336221,-0.307378,-0.331201,-0.324392,-0.325155,-0.337118,-0.317480,-0.298426,-0.017855


In [43]:
## Calling 10 recommended items for item id 
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 5378,
                                    item_dict = movies_dict,
                                    n_items = 5)

Item of interest :Star Wars: Episode II - Attack of the Clones (2002)
Item similar to the above item:
1- Lord of the Rings: The Fellowship of the Ring, The (2001)
2- Minority Report (2002)
3- X2: X-Men United (2003)
4- Spider-Man (2002)
5- Lord of the Rings: The Two Towers, The (2002)
